In [10]:
import librosa as lb
import numpy as np
import tensorflow as tf
import csv
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation
from tensorflow.keras.optimizers import Adam

In [2]:
def extractFeatures(file):
    data, sfreq = lb.load(file)
    mfccFeatures = lb.feature.mfcc(y=data, sr=sfreq, n_mfcc=40)
    features = np.mean(mfccFeatures.T, axis=0)
    return features

In [3]:
features = []
labels = []
for root, directories, files in os.walk('/kaggle/input/turkish-academy-voice-challenge-2023'):
    if "Test" not in root: 
        for file in files:
            fileName = os.path.join(root, file)
            label = root[51]
            data = extractFeatures(fileName)
            features.append(data)
            labels.append(int(label))

In [5]:
x = np.array(features)
labels = np.array(labels)
y = np.zeros((labels.shape[0], 5))
y[np.arange(labels.shape[0]), labels] = 1

xTrain, xVal, yTrain, yVal = train_test_split(x, y, test_size = 0.3, random_state=42)

In [15]:
labelNum = y.shape[1]

model = Sequential([
    Dense(100, input_shape=(xTrain.shape[1],), activation='relu'),
    Dropout(0.5),
    Dense(200, activation='relu'),
    Dropout(0.5),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(y.shape[1], activation='softmax')
])

model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [16]:
model.fit(xTrain, yTrain, batch_size=32, epochs=100, validation_data=(xVal, yVal), verbose=1)

testAccuracy = model.evaluate(xVal, yVal,verbose=0)
print(testAccuracy[1])

Epoch 1/100
67/67 [==============================] - 2s 12ms/step - loss: 52.7123 - accuracy: 0.1905 - val_loss: 6.0172 - val_accuracy: 0.3166
Epoch 2/100
67/67 [==============================] - 1s 11ms/step - loss: 37.9665 - accuracy: 0.2293 - val_loss: 5.0464 - val_accuracy: 0.3679
Epoch 3/100
67/67 [==============================] - 0s 6ms/step - loss: 29.8770 - accuracy: 0.2541 - val_loss: 3.8140 - val_accuracy: 0.4214
Epoch 4/100
67/67 [==============================] - 0s 5ms/step - loss: 23.8131 - accuracy: 0.2845 - val_loss: 2.6551 - val_accuracy: 0.4356
Epoch 5/100
67/67 [==============================] - 0s 5ms/step - loss: 20.1476 - accuracy: 0.2812 - val_loss: 2.2429 - val_accuracy: 0.4301
Epoch 6/100
67/67 [==============================] - 0s 5ms/step - loss: 18.0000 - accuracy: 0.2780 - val_loss: 1.7712 - val_accuracy: 0.4531
Epoch 7/100
67/67 [==============================] - 0s 5ms/step - loss: 14.7895 - accuracy: 0.3014 - val_loss: 1.5893 - val_accuracy: 0.4956
Epoc

In [17]:
testPath = "/kaggle/input/turkish-academy-voice-challenge-2023/Test"

with open('predictions.csv', mode='w') as outputFile:
    outputWriter = csv.writer(outputFile, delimiter=',')
    outputWriter.writerow(["FileName", "Class"])
    
    for root, directories, files in os.walk(testPath):
        for file in files:
            fileName = os.path.join(root, file)
            data = extractFeatures(fileName)
            data = data.reshape(1,-1)
            prediction = (np.argmax(model.predict(data),axis=1))
            outputWriter.writerow([file, prediction[0]])

1/1 [==============================] - 0s 50ms/step
